In [ ]:
import os
import cv2

import numpy as np
import pandas as pd

def weighted_hist(img, weight):
    """
    Compute weighted color histogram on HSV color space.
    """
    # Gaussian blur
    img = cv2.GaussianBlur(img, (9, 9), 0.0)  

    # Convert to HSV color space
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img)

    # Compute Colot Histogram
    hist_h = cv2.calcHist([h], [0], None, [256], [0, 256])
    hist_s = cv2.calcHist([s], [0], None, [256], [0, 256])
    hist_v = cv2.calcHist([v], [0], None, [256], [0, 256])

    # Weighted Average
    hist = weight[0] * hist_h + weight[1] * hist_s + weight[2] * hist_v
    
    return hist
    
def measure_dist(hist, last_hist, return_dist=False, dist=None):
    """
    Compute simialrity and distance between two neightboring frames.
    """
    s = np.sum(np.minimum(hist, last_hist))
    h = np.sum(last_hist)        
    similarity = s/h
    
    if return_dist:
        temp_max = np.maximum(hist, last_hist)
        diff = np.square(hist - last_hist)
        nonzero_index = np.nonzero(temp_max)
        dist = np.sum(diff[nonzero_index]/temp_max[nonzero_index])
        
    return similarity, dist

def HistSimilarity(cap, weight=[0.5,0.3,0.2], skip_frame=15):
    """
    Compute histogram intersection between neighboring frames.
    Output: indices of boundary frames

    Arguments:
        filename: str; 
        weight: list of float; weight of color channels in the order of HSV.
        skip_frame: int; calcualte; Calcualte color histogram every 'skip_frame' frames.
    """
    ret, img = cap.read()
    
    index = -1
    cnt = 0
    last_hist = None
    index_similarity = []
    
    while ret:
        # Skip Frame
        cnt += 1
        if cnt >= skip_frame:
            cnt = 0
        else:
            ret, img = cap.read()
            continue
        
        index += skip_frame
        hist = weighted_hist(img, weight)
        
        if last_hist is not None:
            similarity, _ = measure_dist(hist, last_hist, return_dist=False)
            index_similarity.append([index-skip_frame, similarity]) 
        
        last_hist = hist
        ret, img = cap.read()

    return index_similarity


def DetectBoundary(index_similarity, fps=30, threshold=0.85):
    """
    Return time boundaries of shot (in second).
    Arguements:
        index_similarity: list
        fps: int; number of frames per second
        threshold: float; frames which are less similar to their previous frames are consider to be bondary frames.
    """
    index_similarity = np.array(index_similarity)
    index = np.where(index_similarity[:,1] <= threshold)
    boundaries = (index_similarity[:,0][index]+1)/fps
    return boundaries


def main(filename, result_dir, output_filename):
    os.makedirs(result_dir, exist_ok=True)
    
    cap = cv2.VideoCapture(filename) 
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    index_similarity = HistSimilarity(cap)
    boundaries = DetectBoundary(index_similarity, fps)
    
    # Save as .csv file
    boundaries = pd.DataFrame(boundaries)
    boundaries.to_csv(os.path.join(result_dir, output_filename), index=False)
    return 

if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument('-i', '--fname', dest='filename', type=str, help='input filename')
    parser.add_argument('-d', '--dir', dest='result_dir', type=str, help='path to output files')
    parser.add_argument('-o', '--output', dest='output_filename', type=str, help='output filename')

    args = parser.parse_args()
        
    filename = args.filename or 'cut.mp4'
    result_dir = args.result_dir or './result'
    output_filename = args.output_filename or 'boundaries.csv'
    
    main(filename, result_dir, output_filename)